# Laboratorio 2 - Grafos & Aeropuertos

## Presentado por: Juan Camilo Aguirre, Maria Gabriela Rey

##INICIO - Librerias

In [1]:
import pandas as pd
import folium
from google.colab import drive
import matplotlib.pyplot as plt
from numpy import infty, zeros
import PIL
import urllib
from math import *
from __future__ import annotations
import graphviz as gv
from typing import Any



drive.mount('/content/gdrive')

Mounted at /content/gdrive


## BASE

### SECCION 1: CREACION DE CLASES

In [2]:
#-----SECCION 1: CREACION DE CLASES --------------------------------------------------------------------------------------------------------------------------------------------#
class Grafo:

  #Inicializa las listas de vértices y aristas.
  def __init__( self ):
    self._vertices: list[Grafo.Vertex] = []
    self._edges: list[Grafo.Edge] = []
    self.identifier = 'Code'

  def removeVertex(self, vertex):
        self._vertices.remove(vertex)
        self._edges = [edge for edge in self._edges if edge.vertices[0] != vertex and edge.vertices[1] != vertex]

  # Devuelve una representación en cadena del grafo.
  def __repr__( self ): return f"Grafo: ( \n \t Vertices: { self._vertices }, \n \t Edges: { self._edges } \n)"

  @property
  #Devuelve una copia de la lista de vértices
  def vertices( self ) -> list[Grafo.Vertex] : return self._vertices.copy()

  @property
  #Devuelve una lista de diccionarios que representan los datos de los vértices.
  def data(self) -> list[dict] : return [ v.data for v in self._vertices ]

  @property
  #Devuelve una copia de la lista de aristas.
  def edges( self ) -> list[Grafo.Edge] : return self._edges.copy()

  @property
  #Calcula y devuelve una matriz de costos entre los vértices.
  def costMatrix( self ) -> list[list[int]]:
    vertexNumber = len(self._vertices)
    matrix = [ [ 0 for i in range( vertexNumber ) ] for j in range( vertexNumber ) ]
    for edge in self.edges:
      i, j = self.vertices.index( edge.vertices[0] ), self._vertices.index( edge.vertices[1] )
      matrix[i][j] = matrix[j][i] = edge.weight
    return matrix

  class Vertex:
    def __init__( self, data ):
      self.data: dict = data
      self.identifier = 'Code'
      self.edges: list[Grafo.Edge] = []

    def __repr__( self ):
      return str( self.data[self.identifier] )

    def __lt__( self, vertex ): # <
      for i in vertex.edges:
        if i.vertices[1] == self: return True
      return False

    def __gt__( self, vertex ): # >
      for i in self.edges:
        if i.vertices[1] == vertex: return True
      return False

    def __le__( self, vertex ): # <=
      return self < vertex

    def __ge__( self, vertex ): # =>
      return self > vertex

    def __eq__( self, compare ):
      if isinstance( compare, Grafo.Vertex ) : return self.data == compare.data
      elif isinstance( compare, dict )       : return self.data == compare
      elif isinstance( compare, int )        : return self.data[ self.identifier ] == compare
      return False

    def __hash__(self):
      return hash(str(self.data[self.identifier]))

  class Edge:

    #Inicializa los vértices de origen y destino y el peso de la arista.
    def __init__( self, fromVertex: Grafo.Vertex, toVertex: Grafo.Vertex, weight ):
      self.vertices: tuple(Grafo.Vertex,Grafo.Vertex) = ( fromVertex, toVertex )
      self.weight = weight

    def __repr__( self ):
      return f"( {self.weight}:{self.vertices[0]} <-> {self.vertices[1]} )"

  #Agrega un nuevo vértice al grafo.
  def newVertex( self, data ):
    self._vertices.append( self.Vertex( data ) )

  #Agrega una nueva arista entre dos vértices con un peso dado.
  def newEdge( self, source: str, destination: str, weight: float ):

    source = self.getVertex(source)
    destination = self.getVertex(destination)

    edge = self.Edge( source, destination, weight )

    source.edges.append( edge )
    destination.edges.append( self.Edge( destination, source, weight ) )

    self._edges.append( edge )

  #Obteniene un vértice específico según su valor.
  def getVertex( self, value ):
    left = 0
    right = len( self.vertices ) - 1

    while left <= right:
        mid = left + (right - left) // 2 #índice medio del rango actual.
        vertex = self.vertices[mid].data[self.identifier]
        if vertex == value: return self.vertices[mid]
        elif vertex < value: left = mid + 1
        else: right = mid - 1
    return None

  def addVertex( self, vertex ):
    self._vertices.append( vertex )


  #Calcula la distancia entre dos vértices en coordenadas geográficas.
  def distance( self, source, destination ):

    source = self.getVertex( source )
    destination = self.getVertex( destination )

    earth_radius = 6371

    lt0 = radians(source.data['Latitude'])
    ln0 = radians(source.data['Longitude'])
    ltf = radians(destination.data['Latitude'])
    lnf = radians(destination.data['Longitude'])

    dlt = ltf - lt0
    dln = lnf - ln0

    a = sin( dlt / 2 )**2 + cos(lt0) * cos(ltf) * sin(dln / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return earth_radius * c


  #Obtiene una lista de rutas en el grafo.
  def getRoutes( self ):
    routes = []
    for e in self.edges:
      s = e.vertices[0].data
      d = e.vertices[1].data
      route = {
          'Source': ( s['Latitude'], s['Longitude'] ),
          'Destination': ( d['Latitude'], d['Longitude'] ),
          'Weight': e.weight
          }
      routes.append(route)
    return routes

  #Algoritmo de Dijkstra para encontrar el camino mínimo.
  def dijskstra( self, start ):

    paths = { v: ( infty, None ) for v in self.vertices }
    paths[start] = ( 0, None )

    toVisit = self.vertices
    toVisit.remove( start )

    for e in start.edges:
      paths[ e.vertices[1] ] = ( e.weight, start )

    while toVisit:
      toVisit = sorted( toVisit, key = lambda x: paths[x] ) #
      min = toVisit.pop(0)

      for e in min.edges:
        if paths[min][0] + e.weight < paths[ e.vertices[1] ][0]:
          paths[ e.vertices[1] ] = ( paths[min][0] + e.weight, min )

    return(paths)

  #Determina el camino mínimo entre dos vértices.
  def takePath( self, camminimo, source, destination ):

    if not destination in camminimo or camminimo[destination] == ( infty, None ):
      return f"{source} -/-> {destination} "

    weight = camminimo[destination][0]
    path = f"{destination}"

    while not source == destination:
      destination = camminimo[destination][1]
      path = f"{destination} (>>) " + path

    return f"({round(weight,3)}): " + path

  #Visualización del mapa.
  def display( self ):
    plot = gv.Grafo( comment = "Grafo", engine = 'sfdp' )
    for vertex in self.vertices:
      plot.node( str(vertex) )
    for edge in self.edges:
      plot.edge( str(edge.vertices[0]), str(edge.vertices[1]), label = str(round(edge.weight,3)) )
    return plot

### SECCION 2: LECTURA DE DATOS & SECCION 3: CREACION DE LOGICA DEL GRAFO

In [4]:
#-----SECCION 2: LECTURA DE DATOS ----------------------------------------------------------------------------------------------------------------------------------------------#
drive.mount('/content/gdrive')
data = pd.read_csv('gdrive/My Drive/Uni/excels-csvs/flights_final.csv')


#Crea dos listas vacías para almacenar datos de aeropuertos y rutas.
airports = []
routes = []

for i, row in data.iterrows():

  source = {}
  destination = {}

  for item in dict(row):

    if 'Source' in item: source[ item.replace('Source Airport ', '') ] = row[item]
    elif 'Destination' in item: destination[ item.replace('Destination Airport ', '') ] = row[item]

  if not source in airports: airports.append( source )
  if not destination in airports: airports.append( destination )


  route = (source['Code'],destination['Code'])
  if not ( route in routes or route[::-1] in routes ): routes.append(route)

  #if i > 500: break

#Ordena la lista de aeropuertos según el código del aeropuerto.
airports = sorted(airports, key=lambda x: x['Code'])

#Representar el grafo de rutas entre aeropuertos.
globe = Grafo()

#Agrega cada aeropuerto como un vértice al grafo.
for airport in airports:
  globe.newVertex( airport )

#Itera sobre las rutas y agrega cada ruta como una arista con su distancia calculada.
for route in routes:
  source = globe.getVertex(route[0])
  destination = globe.getVertex(route[1])
  globe.newEdge( route[0], route[1], globe.distance(route[0], route[1]) )


  #-----SECCION 3: CREACION DE LOGICA DE GRAFO -----------------------------------------------------------------------------------------------------------------------------------#

import graphviz as gv

globe = Grafo()

for airport in airports:
  globe.newVertex( airport )

for route in routes:
  source = globe.getVertex(route[0])
  destination = globe.getVertex(route[1])
  globe.newEdge( route[0], route[1], globe.distance(route[0], route[1]) )

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### SECCION 4: CREACION DEL MAPA INTERACTIVO

In [5]:
#-----SECCION 4: CREACION DEL MAPA INTERACTIVO ---------------------------------------------------------------------------------------------------------------------------------#
import folium

map = folium.Map()

airports = globe.data
routes = globe.getRoutes()

for airport in airports:
  latitud = airport['Latitude']
  longitud = airport['Longitude']
  nombre_lugar = ''
  for info in airport:
    nombre_lugar += f"{info}: {airport[info]} \n"

  folium.Marker([latitud, longitud], popup=nombre_lugar).add_to(map)

for route in routes:
  folium.PolyLine( (route['Source'],route['Destination']) , color="red").add_to(map)

#Guardar el mapa en un archivo html.
#NO ABRIR
map.save('mapa_Global.html')

## PUNTO 1

In [6]:
inicio = None
while not inicio:
  inicio = globe.getVertex(input('Por favor, ingrese el código del aeropuerto de salida: ').upper())
  print('')

print("La informacion del aeropuerto dado es:")
print()
for info in inicio.data:
  print(f"{info}: {inicio.data[info]}")
print()

camminimo = globe.dijskstra(inicio)

#Filtra y ordena los caminos mínimos para mostrar primero los más cortos.
camminimo = {
    key: camminimo[key]
    for key in sorted( camminimo, key=lambda x: camminimo[x][0], reverse = True)
    if not camminimo[key][0] == infty
  }

i = 0

#Itera sobre los caminos mínimos y muestra información relacionada con cada camino.
for airport in camminimo:

  #Calcula y almacena el camino mínimo desde 'inicio' al aeropuerto actual.
  cam = globe.takePath( camminimo, inicio, airport )
  caminos = cam[cam.index('(') + 1:cam.index(')')].split(" -> ")

  #Muestra el camino mínimo y la distancia.
  print(">----------------------------------------------------------------------------------------------<")
  print("#",i+1)
  print()

  print('El recorrido es el siguiente:')
  print(globe.takePath(camminimo, inicio, airport))
  print()

  print('La distancia total entre todos los caminos es de :', float(caminos[0]),"km.")
  print()


  print("La informacion del aeropuerto de destino es:")
  print()
  for info in airport.data: #Imprime información sobre el aeropuerto de destino (airport).
    print(f"{info}: {airport.data[info]}")
  print()

  #Limita la impresión a los primeros 10 caminos mínimos.
  i += 1
  if i >= 10: break

Por favor, ingrese el código del aeropuerto de salida: BAQ

La informacion del aeropuerto dado es:

Code: BAQ
Name: Ernesto Cortissoz International Airport
City: Barranquilla
Country: Colombia
Latitude: 10.8896
Longitude: -74.7808

>----------------------------------------------------------------------------------------------<
# 1

El recorrido es el siguiente:
(23157.008): BAQ (>>) BOG (>>) SCL (>>) SYD (>>) PER (>>) XCH (>>) CCK

La distancia total entre todos los caminos es de : 23157.008 km.

La informacion del aeropuerto de destino es:

Code: CCK
Name: Cocos (Keeling) Islands Airport
City: Cocos Keeling Island
Country: Cocos (Keeling) Islands
Latitude: -12.18830013
Longitude: 96.83390045

>----------------------------------------------------------------------------------------------<
# 2

El recorrido es el siguiente:
(22210.663): BAQ (>>) MIA (>>) ATL (>>) NRT (>>) BKI (>>) TWU (>>) TRK (>>) BPN (>>) MDC (>>) SOQ (>>) MKW (>>) DJJ (>>) MKQ

La distancia total entre todos los cami

## PUNTO 2

In [19]:
import folium

# Define gradient colors (adjust as needed)
gradient_colors = ["peru", "saddlebrown", "gold", "orange", "burntorange", "sienna", "red", "maroon"]


llegada = None
while not llegada:
  llegada = globe.getVertex(input('Digite el código del aeropuerto: ').upper())
  print('')

for info in llegada.data:
  print(f"{info}: {llegada.data[info]}")

print()
print(globe.takePath(camminimo, inicio, llegada))

ruta_minima = globe.takePath(camminimo, inicio, llegada)  # Obtiene la ruta mínima como una cadena formateada
codigo_aeropuertos = []  # Inicializa una lista para almacenar los códigos de aeropuertos

# Procesa la cadena de la ruta mínima y extrae los códigos de aeropuertos
if '(' in ruta_minima:
  ruta_minima = ruta_minima[ruta_minima.index('): ') + 3:]  # Recorta la parte antes de los códigos
  codigo_aeropuertos = ruta_minima.split(' (>>) ')  # Divide la cadena en códigos individuales

# Crea un nuevo mapa centrado en la ubicación inicial
map = folium.Map(location=[20, 0], zoom_start=2)

location_ini = None
segment_count = 0  # Track the number of segments
for codigo in codigo_aeropuertos:
  for airport in airports:
    if airport['Code'] == codigo:
      latitud = airport['Latitude']
      longitud = airport['Longitude']
      nombre_lugar = ''
      for info in airport:
        nombre_lugar += f"{info}: {airport[info]} \n"

      # Coloca un marcador en el mapa para el aeropuerto con latitud y longitud
      marker = folium.Marker([latitud, longitud], popup=nombre_lugar)
      marker.add_to(map)

      location_actual = (latitud, longitud)

      if location_ini is not None:
        # Determine color for this segment based on segment count
        color = gradient_colors[segment_count % len(gradient_colors)]
        folium.PolyLine([location_ini, location_actual], color=color).add_to(map)
        segment_count += 1

      # Establece el punto actual como el punto de inicio para el próximo segmento
      location_ini = location_actual

# Guarda el mapa en un archivo HTML
map.save('mapa_ruta_minimo.html')


Digite el código del aeropuerto: NBX

Code: NBX
Name: Nabire Airport
City: Nabire
Country: Indonesia
Latitude: -3.36818
Longitude: 135.496002

(21018.877): BAQ (>>) MIA (>>) ATL (>>) NRT (>>) BKI (>>) TWU (>>) TRK (>>) UPG (>>) AMQ (>>) NBX


## IDEAS

In [ ]:
#Insertar codigo del aeropuerto dado


#PASO 1
#Se buscan todos los caminos que parten de dicho codigo del aeropuerto, acto siguiente, de estos se muestra el recorrido y se muestran en la grafica

In [ ]:
import folium

llegada = None
while not llegada:
  llegada = globe.getVertex(input('Digite el código del aeropuerto: ').upper())
  print('')

for info in llegada.data:
  print(f"{info}: {llegada.data[info]}")

print()
print(globe.takePath( camminimo, inicio, llegada ))

ruta_minima = globe.takePath(camminimo, inicio, llegada)  # Obtiene la ruta mínima como una cadena formateada
codigo_aeropuertos = []  # Inicializa una lista para almacenar los códigos de aeropuertos

# Procesa la cadena de la ruta mínima y extrae los códigos de aeropuertos
if '(' in ruta_minima:
  ruta_minima = ruta_minima[ruta_minima.index('): ') + 3:]  # Recorta la parte antes de los códigos
  codigo_aeropuertos = ruta_minima.split(' -> ')  # Divide la cadena en códigos individuales

# Crea un nuevo mapa centrado en la ubicación inicial
map = folium.Map(location=[20, 0], zoom_start=2)

location_ini = None
for codigo in codigo_aeropuertos:
  for airport in airports:
    if airport['Code'] == codigo:
      latitud = airport['Latitude']
      longitud = airport['Longitude']
      nombre_lugar = ''
      for info in airport:
        nombre_lugar += f"{info}: {airport[info]} \n"

      # Coloca un marcador en el mapa para el aeropuerto con latitud y longitud
      marker = folium.Marker([latitud, longitud], popup=nombre_lugar)
      marker.add_to(map)

      location_actual = (latitud, longitud)

      if location_ini is not None:
        # Cambia el color de la línea a 'red'
        folium.PolyLine([location_ini, location_actual], color="green").add_to(map)

      # Establece el punto actual como el punto de inicio para el próximo segmento
      location_ini = location_actual

# Guarda el mapa en un archivo HTML
map.save('mapa_ruta_minimo.html')


Digite el código del aeropuerto: TIU

Code: TIU
Name: Timaru Airport
City: Timaru
Country: New Zealand
Latitude: -44.30279922
Longitude: 171.2250061

(15516.72): BAQ -> BOG -> SCL -> AKL -> WLG -> TIU


### Mapa de vuelos generado

In [ ]:
import folium
import pandas as pd

# Access the CSV data from your mounted Google Drive
data = pd.read_csv('gdrive/My Drive/Uni/excels-csvs/flights_final.csv')

# Select the first 20 rows (optional, adjust as needed)
data = data.head(25)

# Function to create HTML table for popup content
def create_popup_content(row):
  html = """<table>"""
  for key, value in row.items():
    html += f"<tr><td><b>{key}</b></td><td>{value}</td></tr>"
  html += "</table>"
  return html

# Create the folium map
center_lat = data['Source Airport Latitude'].mean()
center_lon = data['Source Airport Longitude'].mean()
map = folium.Map(location=[center_lat, center_lon], zoom_start=4)  # Adjust zoom level as needed

# Marker and route line creation with error handling
seen_coordinates = set()  # Track used coordinates (optional, to prevent overlap)
for index, row in data.iterrows():
  try:
    source_lat = float(row['Source Airport Latitude'])
    source_lon = float(row['Source Airport Longitude'])
    dest_lat = float(row['Destination Airport Latitude'])
    dest_lon = float(row['Destination Airport Longitude'])
  except:
    print(f"Error processing row: {row}")
    continue  # Skip rows with invalid coordinates

  # Optional: Prevent marker overlap (adjust threshold as needed)
  if (source_lat, source_lon) in seen_coordinates:
    source_lat += 0.01  # Slightly adjust for non-overlapping marker
    source_lon -= 0.01

  # Generate popup content for all airports
  source_popup = create_popup_content(row)
  dest_popup = create_popup_content(row)  # Consistent popup content for both

  source_marker = folium.Marker([source_lat, source_lon], popup=source_popup)
  dest_marker = folium.Marker([dest_lat, dest_lon], popup=dest_popup)
  route_line = folium.PolyLine([
      [source_lat, source_lon],
      [dest_lat, dest_lon]
  ], color="red", weight=2, opacity=0.7)

  map.add_child(source_marker)
  map.add_child(dest_marker)
  map.add_child(route_line)

  seen_coordinates.add((source_lat, source_lon))  # Add used coordinates (optional)

# Save the map as HTML
map.save('mapa_vuelos.html')
